In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr

In [ ]:
consistency_df = pd.read_csv('drive/MyDrive/Colab Notebooks/consistency_df.csv')

In [ ]:
consistency_df

,Unnamed: 0,imageID,C,T,memorability_scores,comments,labels,score
0,0,1019o64,1,4,0.901658,"['tower', 'spire', 'security', 'bitch']","['cloud', 'sky', 'skyscraper', 'building', 'da...",0.250000
1,1,103fe1z,2,38,0.832711,"['kid', 'leg', 'lava', 'hour', 'week', 'bump',...","['water', 'sky', 'azure', 'beach', 'body', 'wa...",0.052632
2,2,1047lma,0,13,0.715258,"['library', 'art', 'videogame', 'library', 'ca...","['shoe', 'furniture', 'chair', 'shelf', 'autom...",0.000000
3,3,10an4jy,0,11,0.757373,"['record', 'player', 'sight', 'friend', 'town'...","['bookcase', 'shelf', 'publication', 'book', '...",0.000000
4,4,10e1w4i,0,6,0.600920,"['view', 'today', 'flight', 'home', 'reason', ...","['cloud', 'skyscraper', 'building', 'atmospher...",0.000000
...,...,...,...,...,...,...,...,...
561,561,t3ffjm,2,19,0.551410,"['castle', 'history', 'chapel', 'hill', 'tradi...","['wheel', 'tire', 'car', 'vehicle', 'property'...",0.105263
562,562,t4vlzu,0,2,0.649913,"['graphic', 'shot']","['people', 'nature', 'art', 'road', 'surface',...",0.000000
563,563,td2msz,0,6,0.932681,"['plumbus', 'grapefruit', 'blood', 'orange', '...","['skyscraper', 'building', 'sky', 'nature', 't...",0.000000
564,564,tersth,0,6,0.446887,"['house', 'end', 'submission', 'rserendipity',...","['automotive', 'design', 'gas', 'circle', 'cam...",0.000000


In [ ]:
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Function to convert string representations of lists into actual lists
def convert_string_to_list(string_list):
    try:
        return ast.literal_eval(string_list)
    except Exception as e:
        print(f"Error converting string to list: {e}")
        return []

# Function to get a word vector from the embeddings
def get_word_vector(word, embeddings_index):
    return embeddings_index.get(word, np.zeros(100))

# Function to compute the average embedding of a list of words
def compute_average_embedding(word_list, embeddings_index):
    vectors = [get_word_vector(word, embeddings_index) for word in word_list]
    if len(vectors) == 0:
        return np.zeros(100)
    return np.mean(vectors, axis=0)

# Function to compute cosine similarity between two lists of words
def compute_similarity(list1, list2, embeddings_index):
    avg_vector1 = compute_average_embedding(list1, embeddings_index)
    avg_vector2 = compute_average_embedding(list2, embeddings_index)
    if np.all(avg_vector1 == 0) or np.all(avg_vector2 == 0):
        return 0  # or handle as needed
    return cosine_similarity([avg_vector1], [avg_vector2])[0][0]


In [ ]:
# Apply this conversion to the entire DataFrame
consistency_df['comments'] = consistency_df['comments'].apply(convert_string_to_list)
consistency_df['labels'] = consistency_df['labels'].apply(convert_string_to_list)


In [ ]:
# Check a few entries to ensure conversion went well
print(consistency_df['comments'].head())
print(consistency_df['labels'].head())


0                      [tower, spire, security, bitch]
1    [kid, leg, lava, hour, week, bump, leg, itch, ...
2    [library, art, videogame, library, card, libra...
3    [record, player, sight, friend, town, time, di...
4       [view, today, flight, home, reason, landscape]
Name: comments, dtype: object
0    [cloud, sky, skyscraper, building, daytime, to...
1    [water, sky, azure, beach, body, water, aqua, ...
2    [shoe, furniture, chair, shelf, automotive, de...
3    [bookcase, shelf, publication, book, wood, she...
4    [cloud, skyscraper, building, atmosphere, sky,...
Name: labels, dtype: object


In [ ]:
glove_file = 'drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_file)

In [ ]:
# Example of computing similarity on processed DataFrame
consistency_df['similarity'] = consistency_df.apply(
    lambda row: compute_similarity(row['comments'], row['labels'], embeddings_index), axis=1)

consistency_df[['comments', 'labels', 'similarity']]


,comments,labels,similarity
0,"[tower, spire, security, bitch]","[cloud, sky, skyscraper, building, daytime, to...",0.804098
1,"[kid, leg, lava, hour, week, bump, leg, itch, ...","[water, sky, azure, beach, body, water, aqua, ...",0.790548
2,"[library, art, videogame, library, card, libra...","[shoe, furniture, chair, shelf, automotive, de...",0.610869
3,"[record, player, sight, friend, town, time, di...","[bookcase, shelf, publication, book, wood, she...",0.546744
4,"[view, today, flight, home, reason, landscape]","[cloud, skyscraper, building, atmosphere, sky,...",0.800182
...,...,...,...
561,"[castle, history, chapel, hill, tradition, con...","[wheel, tire, car, vehicle, property, sky, aut...",0.544203
562,"[graphic, shot]","[people, nature, art, road, surface, wood, tru...",0.463989
563,"[plumbus, grapefruit, blood, orange, sapcicle,...","[skyscraper, building, sky, nature, tower, tow...",0.233809
564,"[house, end, submission, rserendipity, discove...","[automotive, design, gas, circle, camera, lens...",0.720947


In [ ]:
correlation, p_value = spearmanr(consistency_df['memorability_scores'], consistency_df['similarity'])
print(f'Spearman correlation between consistency and memorability: {correlation}, p-value: {p_value}')

Spearman correlation between consistency and memorability: -0.12145239233393722, p-value: 0.0038064029345330676
